<h1>General Algorithm Efficiency</h1>

     

<h4>Considerations when dealing with large files</h4>
Multiprocessing is throwing hardware at the problem.  It's often more effective to optimize the algorithm first.

<h4>Relative speed of operations</h4>
<ul>
    <li>Memory speed is in nanoseconds
    <li>10GbE Network speed is in microseconds (~50)
    <li>Flash speed is in microseconds (between 20-500+)
    <li>Disk speed is in milliseconds (between 4-7)
</ul>
<table width='100%'><tr><td><img src='https://blog.infinio.com/hs-fs/hubfs/relative-speeds-disk-to-dram.png?width=527&name=relative-speeds-disk-to-dram.png'></td><td><img src='https://blog.infinio.com/hs-fs/hubfs/Relative-speeds.png?width=527&name=Relative-speeds.png'></td></tr></table>
       
<a href='https://blog.infinio.com/relative-speeds-from-ram-to-flash-to-disk'>Reference link</a>

<h3>NFS mounts and Internet operations are even slower!</h3><br><br>

<h3>Problem:</h3>
<h4>Compare 5 models using 3 years of monthly ERA5 data.</h4>

<i>Intuitive algorithm:</i>

In [ ]:
for model in models:
    for year in years:
        for month in months:
            #open era5 monthly file ...
            #process data ...
            #write results to file/intermediate data structure ...

    #Aggregate results for model...
#Compare outputs

<br><br><br>
Each monthly file is ~2gb<br>
5 models x 3 years x 12 monthly files = 180 reads of 2gb file from disk = <b>360 gb of data read from disk!</b><br><br><br><br>

<h4>Sometimes the extra complexity of refactoring your code is worth it</h4>
<i>Algorithm 2:</i>

In [ ]:
for year in years:
-->  for month in months:
        #open era5 monthly file ...
-->      for model in models:
            #process data ...
            #write results to separate file/intermediate data structure ...

#Aggregate results for models...
#Compare outputs

3 years x 12 monthly files = 36 reads of 2gb file from disk = <b>72 gb of data read from disk</b><br><br>

<h2>First rule of thumb</h2>
<ul><li><h3>Avoid excessive IO</h3></li></ul><br>

<ul><li><h3>Cache frequently used objects</h3></li></ul>
If an object (file, database lookup, static function output...) are accessed from different places in code and refactoring isn't practical, a simple file cache can improve access dramatically.


<h4>Simple cache</h4>
<ul>
    <li>Declare cache in global names space, usually a dictionary
    <li>Create a wrapper function to access it
        <ul>
            <li>If requested item is in cache, return it
            <li>Otherwise fetch it, load into cache and return it
        </ul>
    <li>If items can go stale, have some way to refresh it
</ul>

<h4>Ex: most recently used file cache</h4>

In [ ]:
from collections import OrderedDict
import pandas as pd

filecache=OrderedDict() #Declare cache in global name space so it remains in memory

def open_cached_file(filename, maxlen=10):#Size max length so all will fit in memory
    """Cache most recently used files"""
    
    if filename in filecache :            #File is already in cache 
        filecache.move_to_end(filename)   #Make this the most recent
        return filecache[filename]        #Return cached version
    
    else :                                #Load from disk
        if len(filecache) >= maxlen :     #See if cache exceeds max length
            t=filecache.popitem(False)    #Remove oldest item (FIFO)
        f=pd.read_csv(filename)           #Read in file  
        #Do initial processing if appropriate.
        filecache[filename]=f             #Store in the cache
        return f                          #Return file

In [ ]:
print("Fetching 1,2,3")
file=open_cached_file('output/t/file1')
file=open_cached_file('output/t/file2')
file=open_cached_file('output/t/file3')
print("Cached files:",', '.join(filecache.keys()),'\n')

print("Fetching 1,2")
file=open_cached_file('output/t/file1')
file=open_cached_file('output/t/file2')
print("Cached files:",', '.join(filecache.keys()),'\n')

print("Fetching 4,5,3")
file=open_cached_file('output/t/file4')
file=open_cached_file('output/t/file5')
file=open_cached_file('output/t/file3')
print("Cached files:",', '.join(filecache.keys()),'\n')

<ul><li><h4>Filter fast</h4></li></ul>
Selectively import data using routines like Pandas read_csv(use_cols=[]) to only import required data<br>
Filter data read from large files first, then process.<br>
<ul><li><h4>Avoid loops, use builtin functions and optimized libraries like Numpy, Pandas & Xarray</h4></ul>
 Loops are run interpreted (slower).<br>Use builtin functions and libraries if available as these are usually compiled and optimized<br>Numpy array functions are vectorized and very fast
 <ul><li><h4>Use dict or set for large lookup table instead of a list.</h4></ul>
 Native dict and set objects use a highly optimized hash lookup table which is O(1)! vs O(n) when searching for membership in a list (<a href='https://wiki.python.org/moin/TimeComplexity'>time complexity</a>)<br>
 <a href='https://www.jessicayung.com/python-lists-vs-dictionaries-the-space-time-tradeoff/'><img src='https://i0.wp.com/www.jessicayung.com/wp-content/uploads/2018/03/ramalho_dictssetslists1.png?w=398'></a>
 

<br><br>
<h3>Other Techniques for Improving Performance and Scalability</h3>
From the Python <a href='https://wiki.python.org/moin/PythonSpeed'>wiki</a>.

Here are some coding guidelines for applications that demand peak performance (in terms of memory utilization, speed, or scalability).<br>

<h4>Use the best algorithms and fastest tools</h4>
<ul>
<li>Membership testing with sets and dictionaries is much faster, O(1), than searching sequences, O(n). When <b>testing "a in b", b should be a set or dictionary instead of a list or tuple</b>.
    <li><b>String concatenation is best done with ''.join(seq)</b> which is an O(n) process. In contrast, using the '+' or '+=' operators can result in an O(n**2) process because new strings may be built for each intermediate step. The CPython 2.4 interpreter mitigates this issue somewhat; however, ''.join(seq) remains the best practice.
<li>Many tools come in both list form and iterator form (range and xrange, map and itertools.imap, list comprehensions and generator expressions, dict.items and dict.iteritems). In general, the iterator forms are more memory friendly and more scalable. They are preferred whenever a real list is not required.
    <li>Many <b>core building blocks are coded in optimized C</b>. Applications that take advantage of them can make substantial performance gains. The building blocks include all of the builtin datatypes (lists, tuples, sets, and dictionaries) and extension modules like array, itertools, and collections.deque.
<li>Likewise, the <b>builtin functions run faster than hand-built equivalents</b>. For example, map(operator.add, v1, v2) is faster than map(lambda x,y: x+y, v1, v2).
<li>Lists perform well as either fixed length arrays or variable length stacks. However, for queue applications using pop(0) or insert(0,v)), collections.deque() offers superior O(1) performance because it avoids the O(n) step of rebuilding a full list for each insertion or deletion.
<li>Custom sort ordering is best performed with Py2.4's key= option or with the traditional decorate-sort-undecorate technique. Both approaches call the key function just once per element. In contrast, sort's cmp= option is called many times per element during a sort. For example, sort(key=str.lower) is faster than sort(cmp=lambda a,b: cmp(a.lower(), b.lower())).
</ul>

<h4>Take advantage of interpreter optimizations</h4>
<ul>
<li>In functions, <b>local variables are accessed more quickly than global variables</b>, builtins, and attribute lookups. So, it is sometimes worth localizing variable access in inner-loops. For example, the code for random.shuffle() localizes access with the line, random=self.random. That saves the shuffling loop from having to repeatedly lookup self.random. Outside of loops, the gain is minimal and rarely worth it.
<li>The previous recommendation is a generalization of the rule to <b>factor constant expressions out of loops</b>. Likewise, constant folding needs to be done manually. Inside loops, write "x=3" instead of "x=1+2".
<li>Function call overhead is large compared to other instructions. Accordingly, it is sometimes worth in-lining code inside time-critical loops.
<li>List comprehensions run a bit faster than equivalent for-loops (unless you're just going to throw away the result).
<li>Starting with Py2.3, the interpreter optimizes while 1 to just a single jump. In contrast, prior to Python 3, while True took several more steps. While the latter was preferred for clarity, time-critical code should have used the first form. Starting in Python 3, True, False, and None are reserved words, so there is no longer any performance difference here. See WhileLoop for additional details.
<li>Multiple assignment is slower than individual assignment. For example "x,y=a,b" is slower than "x=a; y=b". However, multiple assignment is faster for variable swaps. For example, "x,y=y,x" is faster than "t=x; x=y; y=t".
<li>Chained comparisons are faster than using the "and" operator. Write "x < y < z" instead of "x < y and y < z".
<li>A few fast approaches should be considered hacks and reserved for only the most demanding applications. For example, "not not x" is faster than "bool(x)".
</ul>
